In [1]:
# Import statements (standard)
import math
import time
import numpy as np
import pandas as pd
import datetime as dt
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

# Import statements (custom)
import helper_functions as hf

In [2]:
# Load closing price data.
df = pd.read_csv('../cleaned_crypto_closing_prices.csv', index_col='time')
df.index.name = 'time'
df.index = pd.to_datetime(df.index)
df = df[:-1].astype(float) # drop last row (missing closing prices)

In [3]:
df.head()

,BTC_USD,DASH_USD,ETH_USD,LTC_USD,XMR_USD
time,,,,,
2017-09-17 07:00:00,3573.556,294.153333,246.1225,49.590,92.036667
2017-09-17 08:00:00,3595.146,296.706667,246.4850,49.892,92.756667
2017-09-17 09:00:00,3670.806,302.993333,253.5550,51.370,94.123333
2017-09-17 10:00:00,3669.620,301.620000,253.1875,50.940,95.440000
2017-09-17 11:00:00,3688.812,304.310000,255.5600,51.226,95.600000


In [4]:
from sklearn.preprocessing import StandardScaler 

# Create traces.
def create_trace(df, crypto_str, color):
    dates = df.index 
    prices = df[crypto_str].values
    std_prices = (prices - np.mean(prices)) / (np.std(prices, ddof=1))

    trace = go.Scatter(
        x = dates,
        y = std_prices,
        name = crypto_str,
        line = dict(color=color)
    )
    return trace

data = []
colors = ['red', 'green', 'blue', 'orange', 'purple']

for i in range(len(df.columns)):
    col = colors[i]
    s = df.columns[i]
    
    ply_data = create_trace(df, s, col)
    data.append(ply_data)

In [5]:
# Edit the layout, then plot!
layout = dict(title = 'Standardized Closing Prices',
              xaxis = dict(title = 'Date'),
              yaxis = dict(title = 'Standardized Price'),
              )

fig = dict(data=data, layout=layout)
iplot(fig, filename='closing-prices')

### Visualize Custom Features

In [6]:
# Load data (all features).
all_feat_df = pd.read_csv('~/Desktop/Springboard/Cryptocurrency/code/cleaned_crypto_all_features.csv', header=[0,1], skiprows=[2], index_col=0)
all_feat_df.index.name = 'time'
all_feat_df.index = pd.to_datetime(all_feat_df.index)
all_feat_df = all_feat_df[:-1].astype(float) # drop last row (missing closing prices)

In [7]:
# Collapse hierarchical column labels (for sake of ease).
first_level_labels = list(all_feat_df.columns.labels[0])
second_level_labels = list(all_feat_df.columns.labels[1])

new_labels = []

for i in range(len(first_level_labels)):
    f_str = all_feat_df.columns.levels[0][first_level_labels[i]]
    s_str = all_feat_df.columns.levels[1][second_level_labels[i]]
    
    new_str = s_str + '_' + f_str
    new_labels.append(new_str)

all_feat_df.columns = new_labels

In [8]:
# Grab all columns pertaining to Ethereum.
eth_cols = [s for s in list(all_feat_df.columns) if 'ETH' in s]
eth_df = all_feat_df[eth_cols]

In [9]:
# Create traces.
def create_trace(df, color):
    dates = df.index 
    feature = df.values

    trace = go.Scatter(
        x = dates,
        y = feature,
        line = dict(color=color)
    )
    return trace

In [10]:
# Volume plot. 
trace = create_trace(eth_df['ETH_USD_volume'], 'blue')
data = [trace]

layout = dict(title = 'Ethereum Volume',
              xaxis = dict(title = 'Date'),
              yaxis = dict(title = 'Volume'),
              )

fig = dict(data=data, layout=layout)
iplot(fig, filename='volume')

In [11]:
# Fluctuation plot. 
trace = create_trace(eth_df['ETH_USD_fluctuation'], 'blue')
data = [trace]

layout = dict(title = 'Ethereum Fluctuation',
              xaxis = dict(title = 'Date'),
              yaxis = dict(title = 'Fluctuation'),
              )

fig = dict(data=data, layout=layout)
iplot(fig, filename='fluctuation')

In [12]:
# Relative high-low close plot.
trace = create_trace(eth_df['ETH_USD_relative_hl_close'], 'blue')
data = [trace]

layout = dict(title = 'Ethereum Relative High-Low Close',
              xaxis = dict(title = 'Date'),
              yaxis = dict(title = 'Relative High-Low Close'),
              )

fig = dict(data=data, layout=layout)
iplot(fig, filename='rel-hl-close')

### Show Train-Test-Split

In [13]:
price_trace = create_trace(df['ETH_USD'], 'blue')
split = pd.to_datetime('2017/11/30 12:00:00')

vl_trace = go.Scatter(
       x = [split, split],
       y = [df['ETH_USD'].min(), df['ETH_USD'].max()],
       line = dict(color='black', 
                   dash='dash',
                   width=1.0)
    )

data = [price_trace, vl_trace]

In [14]:
layout = dict(title = 'Ethereum (Train-Test Split)',
              xaxis = dict(title = 'Date'),
              yaxis = dict(title = 'Closing Price'),
              showlegend = False
              )

fig = dict(data=data, layout=layout)
iplot(fig, filename='train-test-plot')

### Stationary vs. Non-Stationary

In [15]:
# Create traces.
def create_trace(df, color, label, op, lw):
    dates = df.index 
    feature = df.values
    std_feature = (feature - np.mean(feature)) / (np.std(feature, ddof=1))

    trace = go.Scatter(
        x = dates,
        y = std_feature,
        name = label,
        opacity = op,
        line = dict(color=color, 
                    width=lw)
    )
    return trace

diff_values = df['ETH_USD'].diff().dropna(axis=0)

ns_trace = create_trace(df['ETH_USD'], 'blue', 'Non-Stationary', 1.0, 3.0)
s_trace = create_trace(diff_values, 'red', 'Stationary', 0.33, 1.0)

s_ns_data = [s_trace, ns_trace]

In [16]:
layout = dict(title = 'Ethereum (Stationary vs. Non-Stationary)',
              xaxis = dict(title = 'Date'),
              yaxis = dict(title = 'Standardized Price'),
              )

fig = dict(data=s_ns_data, layout=layout)
iplot(fig, filename='s-ns-plot')